In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver       
from qiskit_nature.second_q.formats import MoleculeInfo
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.optimizers import SLSQP, COBYLA

from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

from qiskit_aer.primitives import Estimator as AerEstimator                                                                          
from qiskit.primitives import BackendEstimator


In [2]:
import numpy as np
import os
import sys

from scipy.optimize import minimize

In [3]:
info = MoleculeInfo(["Al", "H", "H", "H"], [(0.0, 0.0, 0.0),(0.0, 1.58588, 0.0),(1.3733, -0.79293, 0.0),(-1.3734, -0.79296, 0.0)])
driver = PySCFDriver.from_molecule(info, basis="sto-3g")
molecule = driver.run()

In [5]:
as_transformer = ActiveSpaceTransformer(10, 5, active_orbitals=[5, 6, 7, 8, 9])
as_problem = as_transformer.transform(molecule)

hamiltonian = as_problem.hamiltonian.second_q_op()

In [6]:
mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(hamiltonian)

In [23]:
ansatz = TwoLocal(
    num_qubits = qubit_hamiltonian.num_qubits,
    rotation_blocks=["ry", "rz"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=2,
)

#  initial_state = HartreeFock(
#         as_problem.num_spatial_orbitals,
#         as_problem.num_particles,
#         mapper,
#     ),

In [24]:
ideal_estimator = AerEstimator(
    run_options={"shots": 1024},
    approximation=True,
)

In [25]:
num_params = ansatz.num_parameters
num_params

60

In [26]:
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    print(eval_count, len(parameters), mean)

In [30]:
vqe_solver = VQE(ideal_estimator, ansatz, SLSQP(maxiter=2), callback=store_intermediate_result)

In [31]:
calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(as_problem)

1 60 -5.433539775764874
2 60 -5.441653915718874
3 60 -5.408640462392613
4 60 -5.379997308010521
5 60 -5.397766534649447
6 60 -5.404921773464748
7 60 -5.385815447033771
8 60 -5.444347184136293
9 60 -5.410806542908153
10 60 -5.381739338491354
11 60 -5.38326814797032
12 60 -5.424053477972204
13 60 -5.396216540469689
14 60 -5.392024779757257
15 60 -5.425208547527235
16 60 -5.386504836644008
17 60 -5.41630519733007
18 60 -5.357110153556202
19 60 -5.454271566064073
20 60 -5.376218761479486
21 60 -5.407002514899802
22 60 -5.382783183208506
23 60 -5.416245079615985
24 60 -5.429320344143549
25 60 -5.400194435719765
26 60 -5.388555873374347
27 60 -5.379145756541337
28 60 -5.376444689926776
29 60 -5.418034495265641
30 60 -5.426294041114962
31 60 -5.3940102185268675
32 60 -5.424044114985459
33 60 -5.386989728209073
34 60 -5.382448320513337
35 60 -5.39059813533584
36 60 -5.35305336228865
37 60 -5.35318010918475
38 60 -5.3879485352635825
39 60 -5.381135659282265
40 60 -5.35245598719993
41 60 -5.3920

In [32]:
res.eigenvalues

array([-5.47206261])